In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Get user input
merek = input("Masukkan merek kendaraan (contoh: toyota): ")
jenis = input("Masukkan jenis kendaraan (contoh: suv): ")

# Build URL with user input
url = f"https://www.oto.com/cari/mobil-{jenis.replace(' ', '%20')}"
if merek:
    url += f"+{merek.replace(' ', '%20')}"

# Send GET request
response = requests.get(url)

# Check if request successful
if response.status_code == 200:
    # Parse HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all vehicle listing elements
    vehicle_list = soup.find_all("li", class_="card shadow-light filter-listing-card")

    # Initialize list to store vehicle data
    data_mobil = []

    # Iterate over each vehicle element
    for vehicle in vehicle_list:
        # Vehicle Name
        nama_mobil = vehicle.find("a", class_="vh-name").text.strip() if vehicle.find("a", class_="vh-name") else None

        # Vehicle Price
        harga_mobil = vehicle.find("div", class_="vh-price emi m-sm-t m-xs-b").text.split('\n')[1].strip() if vehicle.find("div", class_="vh-price emi m-sm-t m-xs-b") else None

        # Total Vehicle Variants
        varian_mobil = vehicle.find("span", class_="btn-view-all").text.strip() if vehicle.find("span", class_="btn-view-all") else None

        # Vehicle URL
        link = vehicle.find("a", class_="vh-name")["href"] if vehicle.find("a", class_="vh-name") else None

        # Extracting additional details from vehicle link
        if link:
            detail_response = requests.get(link)
            if detail_response.status_code == 200:
                detail_soup = BeautifulSoup(detail_response.content, "html.parser")

                # Extract Rating
                rating_element = detail_soup.find("span", class_="vh-review tag-chip shadow-light")
                rating = float(rating_element.text.split("|")[0]) if rating_element and rating_element.text.strip() != "Tulis Review" else None if rating_element else None

                # Extract Transmission Type
                transmisi_element = detail_soup.find("td", text="Jenis Transmisi")
                transmisi = transmisi_element.find_next_sibling().text.strip().split()[-1] if transmisi_element else None

                # Extract Seating Capacity
                kapasitas_element = detail_soup.find("td", text="Kapasitas Tempat Duduk")
                kapasitas = kapasitas_element.find_next_sibling().text.strip().split()[-2] if kapasitas_element else None

                # Add vehicle data to list
                data_mobil.append({
                    "Vehicle Name": nama_mobil,
                    "Price": harga_mobil,
                    "Vehicle Type": jenis,
                    "Variant": varian_mobil,
                    "Rating": rating,
                    "Transmission Type": transmisi,
                    "Seating Capacity": kapasitas,
                    "Link": link
                })

    # Format table and display
    df = pd.DataFrame(data_mobil)
    print(df.to_string(justify='center'))
    
    # Optional : Save to Excel    
    #df.to_excel("Data Kendaraan Oto Com.xlsx")
    #print("Data kendaraan berhasil disimpan ke Data Kendaraan Oto Com.xlsx")
    
else:
    print("Gagal mengambil data. Kode status:", response.status_code)


Masukkan merek kendaraan (contoh: toyota): Honda
Masukkan jenis kendaraan (contoh: suv): SUV
